# Agent Framework Quickstart

This notebook demonstrates how to create and run a simple AI agent using the Microsoft Agents Framework with Azure OpenAI.

## Prerequisites

Before running this notebook, ensure you have:

- .NET 10.0 SDK or later
- Azure OpenAI service endpoint and deployment configured

This tutorial uses Azure OpenAI for the Chat Completion service, but you can use any inference service that provides an `IChatClient` implementation.

## Step 1: Install Required NuGet Packages

**IMPORTANT**: If you encounter version conflicts, restart the kernel first.

We'll install:
- `Azure.AI.OpenAI` - Azure OpenAI client SDK
- `Azure.Identity` - Authentication support
- `Microsoft.Agents.AI.OpenAI` - Agent Framework for OpenAI

In [ ]:
#r "nuget: Azure.AI.OpenAI, 2.1.0"
#r "nuget: Azure.Identity, 1.17.1"
#r "nuget: DotNetEnv, 3.1.0"
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.260108.1"
#r "nuget: Microsoft.Extensions.AI, 10.2.0"
#r "nuget: Microsoft.Extensions.AI.OpenAI, 10.1.1-preview.1.25612.2"
#r "nuget: Microsoft.Agents.AI.Abstractions, 1.0.0-preview.260108.1"


## Step 2: Load Environment Variables

Load Azure OpenAI configuration from the .env file.

In [ ]:
using DotNetEnv;
using System.IO;

var envFilePath = Path.Combine(Directory.GetCurrentDirectory(), "..", ".env");
if (File.Exists(envFilePath))
{
    Env.Load(envFilePath);
    Console.WriteLine("Loaded environment variables from .env");
}
else
{
    Console.WriteLine($"No .env file found at {envFilePath}");
}

var azureAIEndpoint = Environment.GetEnvironmentVariable("AZURE_AI_FOUNDRY_SERVICE_ENDPOINT");
var chatModelName = Environment.GetEnvironmentVariable("AZURE_CHAT_MODEL_NAME") ?? "gpt-4o";
var apiKey = Environment.GetEnvironmentVariable("AZURE_AI_SERVICES_KEY");

Console.WriteLine($"Azure AI Endpoint: {azureAIEndpoint}");
Console.WriteLine($"Chat Model: {chatModelName}");

## Step 3: Create the Agent

Now we'll create an agent by:
1. Creating an Azure OpenAI client with authenticated credentials
2. Getting a chat client for the specific model deployment
3. Converting it to an AI agent with instructions and a name

In [ ]:
using Azure.AI.OpenAI;
using Azure.Identity;
using Microsoft.Agents.AI;
using Microsoft.Extensions.AI;
using System.ClientModel;

// Create Azure OpenAI client
var azureOpenAIClient = new Azure.AI.OpenAI.AzureOpenAIClient(
        new Uri(azureAIEndpoint), new ApiKeyCredential(apiKey));

// Get chat client for the specified model
var chatClient = azureOpenAIClient.GetChatClient(chatModelName).AsIChatClient();

//create agent
var agent = chatClient.CreateAIAgent(
    instructions: "You are a helpful travel assistant that provides travel recommendations and information.",
    description: "Assists users with travel-related queries such as destination suggestions, itinerary planning, and travel tips.",
    tools: [],
    name: "TravelAssistant");

## Step 4: Run the Agent

Run the agent with a simple string prompt. The agent will return an `AgentResponse` object.

In [ ]:
var response = await agent.RunAsync("What are some must-visit places in Australia?");
Console.WriteLine("Agent Response:");
Console.WriteLine(response.Text);